In [1]:
import numpy as np
from lib.defaults import Main
import json
from lib.controller import dynamic, decompose
from lib.lqr import classical_lqr
from lib.simple import Sim
from scipy.linalg import solve_lyapunov, solve
from scipy.optimize import minimize

In [2]:
# params I need 
main = Main()
a = main.a
r2 = main.r2_dynamic
pmd_slice = main.pmd_slice
taus = main.taus
tau = main.tau
# prms
n = main.n
n_e = main.n_e

with open('setup_prms.json') as f:
    prms = json.load(f)
c = np.array(prms['c'])

In [3]:
tau_z, tau_y = main.taus
q = Sim().q_mov(c)

# I pick out codes from controller.dynamic

In [4]:
n, _ = a.shape
n_pmd = len(pmd_slice)
n_i = n - n_e
tau_z, tau_y = taus
const_y = (tau / tau_y) * np.eye(n_e)
const_z = (tau / tau_z) * np.eye(n_e)

yx = 0.1
x = np.random.binomial(1, yx, size=(n_e, n_e)) / (yx * n_e)
yx = np.hstack((x, np.zeros((n_e, n_i))))

a = np.block([
    [a, np.zeros((n, n_e)), np.zeros((n, n_e))],
    [(tau / tau_y) * yx, -const_y, np.zeros((n_e, n_e))],
    [np.zeros((n_e, n)), const_z, -const_z]
])

b = np.block([
    [np.eye(n)],
    [np.zeros((n_e + n_e, n))]
])

c = np.hstack((np.zeros((n_e, n)), np.zeros((n_e, n_e)), np.eye(n_e)))

q = np.block([
    [q, np.zeros((n, n_e)), np.zeros((n, n_e))],
    [np.zeros((n_e, n)), np.zeros((n_e, n_e)), np.zeros((n_e, n_e))],
    [np.zeros((n_e, n)), np.zeros((n_e, n_e)), np.zeros((n_e, n_e))]
])

# I pick out codes from lqr.output_lqr

In [5]:
n, _ = a.shape
iden = np.eye(n)
clo, raw = (n_pmd, n_e)
npara = clo * raw
bt = b.T
ct = c.T

def k_of(z):
    return z @ c

def s_of(acl):
    return solve_lyapunov(acl, -iden)

def p_of(acl, k):
    return solve_lyapunov(acl.T, -(q + (r2 * k.T @ bt @ b @ k)))

def grad_z(k, p, s):
    return 2 * bt @ (p + (r2 * b @ k)) @ s @ ct   


def f_df(z):
    z = np.reshape(z, (clo, raw))
    k = k_of(z)
    acl = a + (b @ k) 
    p = p_of(acl, k)  
    print(np.trace(p))
    return np.trace(p)

def grad_func(z):
    z = np.reshape(z, (clo, raw))
    k = k_of(z)
    acl = a + (b @ k) 
    s = s_of(acl)
    p = p_of(acl, k)
    g_ = grad_z(k, p, s)    
    return g_.reshape(npara)

z0 = classical_lqr(r2, q, a, b) @ solve(c @ c.T, c).T
# fmin_l_bfgs_b(f_df, z0.reshape(npara),fprime=grad_func)
res = minimize(f_df, z0.reshape(npara), jac=grad_func, method='L-BFGS-B')
# res = minimize(f_df, z0.reshape(npara), jac=grad_func, method='BFGS')
#     print(res.fun)

503.29366696359415
878.5045822062085
484.7069701344527
456.29354438172123
432.3216246258391
402.97722436332464
362.5516034125618
317.9707802632353
281.25164688222435
261.47183917108697
234.51995560607156
229.04690614803536
209.08694724497244
202.64712585365825
194.49518891698963
184.97402235264863
191.76918666518927
180.3172886004208
172.47346841124096
168.02489162138363
163.7103356775324
160.08521570012437
157.87985795968837
155.1169511561639
152.68564652782945
150.60518793490138
149.16960078637388
147.52628804525938
146.41564826200033
145.34790190343563
144.71507973237172
143.5110001009477
142.8814115367305
142.09392935953
141.58506960840077
141.15288640799213
140.58844954868144
140.07934924005434
139.76678618182595
139.27152281496097
139.08653190794854
138.84398952178236
138.51585603052632
138.32706542413644
137.98930729058696
137.78957896679077
137.60041353797405
137.4894237691052
137.367347430452
137.079955874871
137.34493209084008
136.96623398611672
136.7947961754826
136.64204011

133.7872973606937
133.78725250564906
133.7872118309023
133.78717753626074
133.78713301117944
133.78705373917296
133.78699966138285
133.78693079588015
133.78688735322308
133.78684431002537
133.78681149816072
133.78678285738138
133.78670500260586
133.78666942270024
133.78659180047654
133.78655457315432
133.78652532272804
133.7864654521394
133.78639093923303
133.7863464972649
133.78630946081395
133.78627471013164
133.78620984373453
133.78615060297315
133.78611701207058
133.78605295516059
133.78601508882846
133.7859623776152
133.7859316256928
133.78589410915674
133.78584701110066
133.7857936217166
133.7857713796845
133.7857447357457
133.7857285903473
133.78568488781406
133.7856535971102
133.78563559227115
133.78559918864056
133.78554707235668
133.7855067047263
133.78547588006307
133.78544648114922
133.7854160970124
133.78539630451115
133.78535760906826
133.78533973537208
133.78532317795168
133.78528769695282
133.78530440104902
133.78526865178262
133.78523062748977
133.78520165587153
133.78